# Variational AutoEncoder

## Import

In [102]:
import learning2read
learning2read.reload_all()
from learning2read.b05 import *

## Load data

In [103]:
raw_train = Data('btrain')
raw_test = Data('btest')
raw_implicit = Data('implicit')
raw_user = Data('user')
raw_book = Data('book')
raw_dataset = {
    'raw_train' : raw_train,
    'raw_test' : raw_test,
    'raw_implicit' : raw_implicit,
    'raw_user' : raw_user,
    'raw_book' : raw_book,
}


In [104]:
df_total = learning2read.preprocessing.TotalDataFrame.run([raw_train, raw_test, raw_implicit])['output']

In [105]:
dfg_user=df_total.groupby('User-ID')
gdf_user=dfg_user.agg({'Book-Rating':['count','min','max']})
gdf_user.columns=['count','min','max'] # cancel multilevel index
gdf_user_gte10=gdf_user.loc[gdf_user['count']>=1000,'count'].sort_values(ascending=False)
print(len(gdf_user_gte10))
gdf_user_gte10[:5]

117


User-ID
188ec05cf3    13602
58a34dcfe5     7550
91a5b876c9     6109
997dc62862     5891
67a302acdf     5850
Name: count, dtype: int64

In [106]:
gdf_book=df_total.groupby('ISBN').agg({'User-ID':'count'})
gdf_book.columns=['count']
gdf_book=gdf_book.loc[gdf_book['count']>=2,:] # cut
# gdf_book=gdf_book.loc[gdf_book['count']>=300,:] # cut2
gdf_book=gdf_book.sort_values('count',ascending=False)
dim1=len(gdf_book.index)
dim2=len(gdf_user_gte10.index)
dim1,dim2

(143511, 117)

In [107]:
from collections import defaultdict
user_vector_id = defaultdict(lambda: -1)
book_id = defaultdict(lambda: -1)
i=0
for x in gdf_user_gte10.index:
    user_vector_id[x] = i
    i+=1
i=0
for x in gdf_book.index:
    book_id[x]=i
    i+=1

index_list = []
for r in df_total.to_dict('record'):
    uid = user_vector_id[r['User-ID']]
    bid = book_id[r['ISBN']]
    if uid>=0 and bid>=0:
        index_list.append([bid,uid])

## Parsing CLI arguements

In [108]:
import argparse
parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=True,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args([])
args.cuda = not args.no_cuda and torch.cuda.is_available()
# Sets the seed for generating random numbers. Returns a torch._C.Generator object.
torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

In [109]:
index_tns=torch.LongTensor(index_list).t()
value_tns=torch.ones(index_tns.size(1))
train_tns=torch.sparse.FloatTensor(index_tns, value_tns, torch.Size([dim1,dim2]))

In [110]:
train_tns=train_tns.to_dense()

In [111]:
train_tns.size()

torch.Size([143511, 117])

In [112]:
class BookVectorData(torch.utils.data.Dataset):
    """

    Args:
        root (string): Root directory of dataset where ``processed/training.pt``
            and  ``processed/test.pt`` exist.
        train (bool, optional): If True, creates dataset from ``training.pt``,
            otherwise from ``test.pt``.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
    """

    def __init__(self, train_features):
        self.train_features = train_features

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: target_data[index]
        """
        target = self.train_features[index]
        return target

    def __len__(self):
        return len(self.train_features)

In [113]:
train_dataset = BookVectorData(train_tns)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size, shuffle=True, **kwargs)

In [114]:
from torch import nn, optim
from torch.nn import functional as F
import datetime
now=datetime.datetime.now

In [115]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(117, 64)
        self.fc21 = nn.Linear(64, 16)
        self.fc22 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 64)
        self.fc4 = nn.Linear(64, 117)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 117))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [116]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [117]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 117), size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [118]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [119]:
for epoch in range(1, args.epochs + 1):
    train(epoch)

Train Epoch: 1 [0/143511 (0%)]	Loss: 83.534233
Train Epoch: 1 [1280/143511 (1%)]	Loss: 73.687332
Train Epoch: 1 [2560/143511 (2%)]	Loss: 64.140427
Train Epoch: 1 [3840/143511 (3%)]	Loss: 54.942703
Train Epoch: 1 [5120/143511 (4%)]	Loss: 43.331375
Train Epoch: 1 [6400/143511 (4%)]	Loss: 33.082108
Train Epoch: 1 [7680/143511 (5%)]	Loss: 24.876284
Train Epoch: 1 [8960/143511 (6%)]	Loss: 21.853683
Train Epoch: 1 [10240/143511 (7%)]	Loss: 15.293964
Train Epoch: 1 [11520/143511 (8%)]	Loss: 12.673884
Train Epoch: 1 [12800/143511 (9%)]	Loss: 14.842859
Train Epoch: 1 [14080/143511 (10%)]	Loss: 13.294661
Train Epoch: 1 [15360/143511 (11%)]	Loss: 11.352051
Train Epoch: 1 [16640/143511 (12%)]	Loss: 10.108412
Train Epoch: 1 [17920/143511 (12%)]	Loss: 11.461728
Train Epoch: 1 [19200/143511 (13%)]	Loss: 10.398964
Train Epoch: 1 [20480/143511 (14%)]	Loss: 10.211489
Train Epoch: 1 [21760/143511 (15%)]	Loss: 8.846856
Train Epoch: 1 [23040/143511 (16%)]	Loss: 10.780325
Train Epoch: 1 [24320/143511 (17%)]

Train Epoch: 2 [62720/143511 (44%)]	Loss: 5.029554
Train Epoch: 2 [64000/143511 (45%)]	Loss: 7.016000
Train Epoch: 2 [65280/143511 (45%)]	Loss: 5.635261
Train Epoch: 2 [66560/143511 (46%)]	Loss: 5.945562
Train Epoch: 2 [67840/143511 (47%)]	Loss: 4.993707
Train Epoch: 2 [69120/143511 (48%)]	Loss: 5.427718
Train Epoch: 2 [70400/143511 (49%)]	Loss: 5.219595
Train Epoch: 2 [71680/143511 (50%)]	Loss: 6.812445
Train Epoch: 2 [72960/143511 (51%)]	Loss: 4.978668
Train Epoch: 2 [74240/143511 (52%)]	Loss: 5.743450
Train Epoch: 2 [75520/143511 (53%)]	Loss: 7.197273
Train Epoch: 2 [76800/143511 (53%)]	Loss: 5.004735
Train Epoch: 2 [78080/143511 (54%)]	Loss: 6.842177
Train Epoch: 2 [79360/143511 (55%)]	Loss: 6.333487
Train Epoch: 2 [80640/143511 (56%)]	Loss: 5.433034
Train Epoch: 2 [81920/143511 (57%)]	Loss: 6.173864
Train Epoch: 2 [83200/143511 (58%)]	Loss: 7.439878
Train Epoch: 2 [84480/143511 (59%)]	Loss: 4.963384
Train Epoch: 2 [85760/143511 (60%)]	Loss: 6.219549
Train Epoch: 2 [87040/143511 (6

Train Epoch: 3 [124160/143511 (86%)]	Loss: 5.133265
Train Epoch: 3 [125440/143511 (87%)]	Loss: 4.609357
Train Epoch: 3 [126720/143511 (88%)]	Loss: 6.624432
Train Epoch: 3 [128000/143511 (89%)]	Loss: 5.876292
Train Epoch: 3 [129280/143511 (90%)]	Loss: 6.722567
Train Epoch: 3 [130560/143511 (91%)]	Loss: 5.423199
Train Epoch: 3 [131840/143511 (92%)]	Loss: 6.385238
Train Epoch: 3 [133120/143511 (93%)]	Loss: 7.321950
Train Epoch: 3 [134400/143511 (94%)]	Loss: 6.019308
Train Epoch: 3 [135680/143511 (94%)]	Loss: 5.963578
Train Epoch: 3 [136960/143511 (95%)]	Loss: 6.700415
Train Epoch: 3 [138240/143511 (96%)]	Loss: 5.906267
Train Epoch: 3 [139520/143511 (97%)]	Loss: 6.367434
Train Epoch: 3 [140800/143511 (98%)]	Loss: 5.205314
Train Epoch: 3 [142080/143511 (99%)]	Loss: 5.074015
Train Epoch: 3 [143360/143511 (100%)]	Loss: 6.063494
====> Epoch: 3 Average loss: 6.2945
Train Epoch: 4 [0/143511 (0%)]	Loss: 6.901792
Train Epoch: 4 [1280/143511 (1%)]	Loss: 7.673475
Train Epoch: 4 [2560/143511 (2%)]	Lo

Train Epoch: 5 [39680/143511 (28%)]	Loss: 6.081035
Train Epoch: 5 [40960/143511 (29%)]	Loss: 6.316836
Train Epoch: 5 [42240/143511 (29%)]	Loss: 7.112247
Train Epoch: 5 [43520/143511 (30%)]	Loss: 6.007400
Train Epoch: 5 [44800/143511 (31%)]	Loss: 6.146648
Train Epoch: 5 [46080/143511 (32%)]	Loss: 6.690079
Train Epoch: 5 [47360/143511 (33%)]	Loss: 6.551326
Train Epoch: 5 [48640/143511 (34%)]	Loss: 7.014645
Train Epoch: 5 [49920/143511 (35%)]	Loss: 5.802001
Train Epoch: 5 [51200/143511 (36%)]	Loss: 6.383507
Train Epoch: 5 [52480/143511 (37%)]	Loss: 5.114145
Train Epoch: 5 [53760/143511 (37%)]	Loss: 5.207792
Train Epoch: 5 [55040/143511 (38%)]	Loss: 6.650043
Train Epoch: 5 [56320/143511 (39%)]	Loss: 6.741393
Train Epoch: 5 [57600/143511 (40%)]	Loss: 4.983960
Train Epoch: 5 [58880/143511 (41%)]	Loss: 6.413943
Train Epoch: 5 [60160/143511 (42%)]	Loss: 6.316024
Train Epoch: 5 [61440/143511 (43%)]	Loss: 6.129595
Train Epoch: 5 [62720/143511 (44%)]	Loss: 6.530789
Train Epoch: 5 [64000/143511 (4

Train Epoch: 6 [103680/143511 (72%)]	Loss: 6.295475
Train Epoch: 6 [104960/143511 (73%)]	Loss: 6.639312
Train Epoch: 6 [106240/143511 (74%)]	Loss: 4.707234
Train Epoch: 6 [107520/143511 (75%)]	Loss: 6.299406
Train Epoch: 6 [108800/143511 (76%)]	Loss: 6.815423
Train Epoch: 6 [110080/143511 (77%)]	Loss: 6.118813
Train Epoch: 6 [111360/143511 (78%)]	Loss: 5.840068
Train Epoch: 6 [112640/143511 (78%)]	Loss: 5.129815
Train Epoch: 6 [113920/143511 (79%)]	Loss: 5.828417
Train Epoch: 6 [115200/143511 (80%)]	Loss: 5.385818
Train Epoch: 6 [116480/143511 (81%)]	Loss: 7.631042
Train Epoch: 6 [117760/143511 (82%)]	Loss: 7.041106
Train Epoch: 6 [119040/143511 (83%)]	Loss: 6.305265
Train Epoch: 6 [120320/143511 (84%)]	Loss: 5.645521
Train Epoch: 6 [121600/143511 (85%)]	Loss: 7.478806
Train Epoch: 6 [122880/143511 (86%)]	Loss: 5.422095
Train Epoch: 6 [124160/143511 (86%)]	Loss: 6.205401
Train Epoch: 6 [125440/143511 (87%)]	Loss: 5.574083
Train Epoch: 6 [126720/143511 (88%)]	Loss: 7.215813
Train Epoch:

Train Epoch: 8 [21760/143511 (15%)]	Loss: 5.798108
Train Epoch: 8 [23040/143511 (16%)]	Loss: 5.137659
Train Epoch: 8 [24320/143511 (17%)]	Loss: 5.713527
Train Epoch: 8 [25600/143511 (18%)]	Loss: 5.332899
Train Epoch: 8 [26880/143511 (19%)]	Loss: 5.290483
Train Epoch: 8 [28160/143511 (20%)]	Loss: 5.852433
Train Epoch: 8 [29440/143511 (20%)]	Loss: 5.082150
Train Epoch: 8 [30720/143511 (21%)]	Loss: 5.288410
Train Epoch: 8 [32000/143511 (22%)]	Loss: 5.397590
Train Epoch: 8 [33280/143511 (23%)]	Loss: 6.288931
Train Epoch: 8 [34560/143511 (24%)]	Loss: 6.645655
Train Epoch: 8 [35840/143511 (25%)]	Loss: 6.538800
Train Epoch: 8 [37120/143511 (26%)]	Loss: 6.336383
Train Epoch: 8 [38400/143511 (27%)]	Loss: 6.117859
Train Epoch: 8 [39680/143511 (28%)]	Loss: 6.249860
Train Epoch: 8 [40960/143511 (29%)]	Loss: 6.994263
Train Epoch: 8 [42240/143511 (29%)]	Loss: 5.209042
Train Epoch: 8 [43520/143511 (30%)]	Loss: 5.772225
Train Epoch: 8 [44800/143511 (31%)]	Loss: 6.154293
Train Epoch: 8 [46080/143511 (3

Train Epoch: 9 [84480/143511 (59%)]	Loss: 4.895270
Train Epoch: 9 [85760/143511 (60%)]	Loss: 5.947866
Train Epoch: 9 [87040/143511 (61%)]	Loss: 6.249286
Train Epoch: 9 [88320/143511 (61%)]	Loss: 7.531250
Train Epoch: 9 [89600/143511 (62%)]	Loss: 5.687867
Train Epoch: 9 [90880/143511 (63%)]	Loss: 5.506376
Train Epoch: 9 [92160/143511 (64%)]	Loss: 5.833853
Train Epoch: 9 [93440/143511 (65%)]	Loss: 5.698204
Train Epoch: 9 [94720/143511 (66%)]	Loss: 6.559685
Train Epoch: 9 [96000/143511 (67%)]	Loss: 6.959698
Train Epoch: 9 [97280/143511 (68%)]	Loss: 5.089138
Train Epoch: 9 [98560/143511 (69%)]	Loss: 7.299997
Train Epoch: 9 [99840/143511 (70%)]	Loss: 5.046866
Train Epoch: 9 [101120/143511 (70%)]	Loss: 6.098492
Train Epoch: 9 [102400/143511 (71%)]	Loss: 6.345285
Train Epoch: 9 [103680/143511 (72%)]	Loss: 5.425423
Train Epoch: 9 [104960/143511 (73%)]	Loss: 6.797670
Train Epoch: 9 [106240/143511 (74%)]	Loss: 6.457715
Train Epoch: 9 [107520/143511 (75%)]	Loss: 7.965286
Train Epoch: 9 [108800/14

Train Epoch: 10 [143360/143511 (100%)]	Loss: 6.285866
====> Epoch: 10 Average loss: 6.0967
